In [87]:
Exch1_2 = {"tokens": "1 to 2", "a_in": 1, "a_out": 2}
Exch2_3 = {"tokens": "2 to 3", "a_in": 1, "a_out": 2}
Exch3_4 = {"tokens": "3 to 4", "a_in": 1, "a_out": 2}
Exch4_1 = {"tokens": "4 to 1", "a_in": 1, "a_out": 2}
r = 0.9

Trades = [Exch1_2, Exch2_3, Exch3_4, Exch4_1]

In [88]:
def outgoing(din, r, ain, aout):
    dout = (aout * r * din) / (ain + r * din)
    return dout


In [89]:
def do_trades(Trades, d_begin, r):
    d = [d_begin]
    for i in range(len(Trades)):
        a_in = Trades[i]['a_in']
        a_out = Trades[i]['a_out']
        dout = outgoing(d[i], r, a_in, a_out)
        print(f"Tokens {Trades[i]["tokens"]} \t Incoming leg: {d[i]:6.4f}, outgoing leg:{dout:6.4f} ")
        d.append(dout)
        din = dout
    print(f"Profit: {d[-1] - d[0]:6.4f}")

In [90]:
do_trades(Trades, 0.01, r)

Tokens 1 to 2 	 Incoming leg: 0.0100, outgoing leg:0.0178 
Tokens 2 to 3 	 Incoming leg: 0.0178, outgoing leg:0.0316 
Tokens 3 to 4 	 Incoming leg: 0.0316, outgoing leg:0.0553 
Tokens 4 to 1 	 Incoming leg: 0.0553, outgoing leg:0.0948 
Profit: 0.0848


In [91]:
# Equiv liquidity (algorithm 1)
Exchprime = {"tokens": "4 to 1", "a_in": 1, "a_out": 2}
Exchprime["a_out"] = Trades[0]['a_in']
Exchprime["a_in"]  = Trades[0]['a_out']
for i in range(2, len(Trades)):
    AprimeOut = Exchprime['a_out']
    AprimeIn = Exchprime['a_in']
    Exchprime["a_out"] = AprimeOut * Trades[i]["a_in"]/(Trades[i]["a_in"] + r * AprimeIn )
    Exchprime["a_in"]  = r * AprimeIn  * Trades[i]["a_out"]/(Trades[i]["a_in"] + r * AprimeIn)
    print(f"Prime tokens {Exchprime["tokens"]} \t a_in {Exchprime["a_in"]} \t a_out {Exchprime["a_out"]} ")



Prime tokens 4 to 1 	 a_in 1.2857142857142858 	 a_out 0.35714285714285715 
Prime tokens 4 to 1 	 a_in 1.0728476821192052 	 a_out 0.16556291390728475 


In [92]:
# Optimal
a = (Exchprime["a_out"] * Trades[-1]["a_in"]) / (Trades[-1]["a_in"] + r * Exchprime["a_in"])
aprime = (r * Trades[-1]["a_out"] * Exchprime["a_in"] ) / (Trades[-1]["a_in"] + r * Exchprime["a_in"])

In [93]:
import numpy as np
dopt = (np.sqrt(r * aprime * a) - a )/r

In [94]:
print(f"Optimal starting trade: {dopt:6.4f}")

Optimal starting trade: 0.2096


In [97]:
oldprofit = 0
olddin = 0
for i in range(1,20):
    din = dopt - 0.005 + i * 0.001
    d = [din]


    for i in range(len(Trades)):
        dout = outgoing(din, r, Trades[i]["a_in"], Trades[i]["a_out"])
        #print(f"Incoming leg: {din:6.4f}, outgoing leg:{dout:6.4f} ")
        d.append(dout)
        din = dout
    profit = d[-1] - d[0]

    if profit > oldprofit:
        oldprofit = profit
        olddin = d[0]
        print(f"Starting trade: {d[0]:6.4f} \t Profit: {d[-1]-d[0]:6.4f}  ")
    else:
        print(f"Optimal!")
        if dopt == olddin:
            print(f"Starting trade dopt: {dopt:6.4f} \t Profit {oldprofit:6.4f} ")
        else:
            print(f"Starting trade was not dopt but: {d[0]:6.4f} \t Profit {oldprofit:6.4f} ")
        break



Starting trade: 0.2056 	 Profit: 0.4695  
Starting trade: 0.2066 	 Profit: 0.4696  
Starting trade: 0.2076 	 Profit: 0.4696  
Starting trade: 0.2086 	 Profit: 0.4696  
Starting trade: 0.2096 	 Profit: 0.4696  
Optimal!
Starting trade dopt: 0.2096 	 Profit 0.4696 
